<a href="https://colab.research.google.com/github/neotracer1/ATENEA_UNAL_403_2025/blob/main/GEE_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este código importará imagenes de GEE para el proyecto de huemedales ATENEA 🗺

## Primero se instala geemap y se importan librerias

In [6]:
!pip -q install geemap


In [7]:
import ee
import geopandas as gpd
import geemap

## Se usa la API de GEE para autentificarse e inicializar el proyecto

In [9]:

ee.Authenticate()

In [10]:
# Initialize the library.
ee.Initialize(project='ee-wiamartinezma97') ##Acá se debe ajustar el nombre del proyecto según el usuario de GEE

## Se importan los ROI del área de estudio

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
shp_path    = "/content/drive/MyDrive/UNAL_MIA/ATENEA/SIG/SHAPE/ROI_BOGOTÁ.shp"

In [13]:
# --- Cargar ROI y ver el CRS ---
roi = gpd.read_file(shp_path)
roi.crs #Se debe validar el CRS ya que en GEE solo se trabaja con 4326

<Projected CRS: PROJCS["CTM12",GEOGCS["MAGNA-SIRGAS",DATUM["Marco_ ...>
Name: CTM12
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: Marco Geocentrico Nacional de Referencia
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [14]:

roi_gdf = roi.to_crs(4326)  # Earth Engine espera lon/lat (EPSG:4326)

# Unir todo en una sola geometría (por si tienes varios polígonos)
roi_union = roi_gdf.unary_union

# Pasar a GeoJSON y luego a ee.Geometry
roi = ee.Geometry(roi_union.__geo_interface__)

/tmp/ipython-input-1776635184.py:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  roi_union = roi_gdf.unary_union


## Se cran funciones para nubosidad e indices espectrales



In [15]:
#Función de nubosidad

def mask_s2_clouds(image: ee.Image) -> ee.Image:
    qa = image.select("QA60")
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
        qa.bitwiseAnd(cirrus_bit_mask).eq(0)
    )

    return (
        image.updateMask(mask)
        .divide(10000)
        .copyProperties(image, image.propertyNames())
    )

#Función de indices espectrales

def add_indices(img: ee.Image) -> ee.Image:
    nir = img.select("B8")
    red = img.select("B4")
    green = img.select("B3")
    blue = img.select("B2")
    swir1 = img.select("B11")

    ndvi = nir.subtract(red).divide(nir.add(red)).rename("NDVI")
    mndwi = green.subtract(swir1).divide(green.add(swir1)).rename("MNDWI")
    ndmi = nir.subtract(swir1).divide(nir.add(swir1)).rename("NDMI")
    ndbi = swir1.subtract(nir).divide(swir1.add(nir)).rename("NDBI")

    evi = (
        nir.subtract(red)
        .multiply(2.5)
        .divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1))
        .rename("EVI")
        .toFloat()
    )

    L = 0.5
    savi = (
        nir.subtract(red)
        .multiply(1 + L)
        .divide(nir.add(red).add(L))
        .rename("SAVI")
        .toFloat()
    )

    return img.addBands([ndvi, evi, savi, mndwi, ndmi, ndbi])


## Se importa la colección de S2, se ejecuta cada función y se crea un mosaico apartir de la mediana

In [16]:


bands_out = [
    "B2", "B3", "B4", "B5", "B6", "B7",
    "NDVI", "EVI", "SAVI", "MNDWI", "NDMI", "NDBI"
]

dataset = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2024-01-01", "2024-12-31")
    .filterBounds(roi)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .map(mask_s2_clouds)
    .map(add_indices)
    .map(lambda img: img.select(bands_out))
)

mosaic = dataset.median().toFloat().clip(roi)

# Ver nombres de bandas
print("Bandas finales:", mosaic.bandNames().getInfo())




Bandas finales: ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI', 'EVI', 'SAVI', 'MNDWI', 'NDMI', 'NDBI']


## Se hace una visualización del mosaico y los indices agregados

In [17]:

Map = geemap.Map(center=[4.65, -74.1], zoom=10)

# RGB
rgb_vis = {"bands": ["B4","B3","B2"], "min": 0.0, "max": 0.5, "gamma": 1.2}
Map.addLayer(mosaic, rgb_vis, "RGB SR")

# NDVI: rojo (-1) -> verde (1)
Map.addLayer(
    mosaic.select("NDVI"),
    {"min": -1, "max": 1, "palette": ["red", "yellow", "green"]},
    "NDVI"
)

# MNDWI: blanco -> azul
Map.addLayer(
    mosaic.select("MNDWI"),
    {"min": -1, "max": 1, "palette": ["white", "blue"]},
    "MNDWI"
)

# NDBI: blanco -> rojo
Map.addLayer(
    mosaic.select("NDBI"),
    {"min": -1, "max": 1, "palette": ["white", "red"]},
    "NDBI"
)

# ROI (si roi es ee.Geometry)
Map.addLayer(ee.FeatureCollection([ee.Feature(roi)]), {}, "ROI")

Map


Map(center=[4.65, -74.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

## FInalmente luego de verificar los resultados se hace el proceso de exportación de la imagen a drive

In [18]:
task = ee.batch.Export.image.toDrive(
    image=mosaic,
    description="S2_BOGOTA_2024",
    folder="ATENEA",  # DRIVE SOLO PERMITE EXPORTAR A CARPETAS RAIZ, SE DEBE MOVER ESTA A DONDE SE TIENE LA ESTRUCTURA DE LA INFORMACIÓN
    fileNamePrefix="S2_BOGOTA_2024",
    region=roi,
    scale=10,
    maxPixels=1e13,
    fileFormat="GeoTIFF"
)
task.start()

Ver estado de la exportación, es importante esperar al menos 10 minutos luego de ejecutar el script anterior

In [ ]:

print("ID:", task.id)
print(task.status())
print("Estado:", task.status()["state"])


ID: TEV4EU2KNZR7HO3BWAWH4E5L
{'state': 'COMPLETED', 'description': 'S2_BOGOTA_2024', 'priority': 100, 'creation_timestamp_ms': 1765146263467, 'update_timestamp_ms': 1765146493595, 'start_timestamp_ms': 1765146268346, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/17lWAoDemiraFa9n2nOQBDNehh_cpDfxQ'], 'attempt': 1, 'batch_eecu_usage_seconds': 461.4183349609375, 'id': 'TEV4EU2KNZR7HO3BWAWH4E5L', 'name': 'projects/ee-wiamartinezma97/operations/TEV4EU2KNZR7HO3BWAWH4E5L'}
Estado: COMPLETED
